## Create Stock Price DataBase with Python & SQL

### Contents:

1. Create functions
2. Dow Jones Industrial Average (DJIA)
    - Scrape ticker lists from Wikipedia 
    - Download historical data from from Yahoo! Finance's API 
3. S&P/TSX60 
    - Scrape ticker lists from Wikipedia
    - Cleaning data for ticker lists to meet Yahoo! Finance's API
    - Download historical data from from Yahoo! Finance's API
4. Download fundamental data from Yahoo! Finance's API
5. Market information
    - Download historical market data for some important indexes
6. Create and import data into SQLite database

I am going to use this data for building a financial Analysis Dashboard on Power Bi and answer some questions about financial analysis using SQL

In [1]:
# Import important packages
import sqlalchemy
import yfinance as yf
import pandas as pd

##### The yfinance Ticker object:
* _start_ and _end_ dates
* _period_ instead of start/end
* valid periods: 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y, ytd, max
* valid intervals: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1w, 1mo, 3mo
* _actions=True_: (Optional) inclues info on both dividends and splits.
* _prepost=True_: (Optional) download pre/post regular market hours data

## 1. Create Functions

In [2]:
# Function to collect data
def getData(tickers):
    data = []
    for ticker in tickers:
        data.append(yf.download(ticker, period='max').reset_index())
    return data


#Function to create symbol column and merge multiple data into one dataframe
def merge_data(frames, symbols):
    for i in range(len(frames)):
        frames[i]['Symbol'] = symbols[i]
    newframes = [df.set_index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Symbol']) for df in frames]
    merged_df = pd.concat(newframes, axis=1).reset_index()
    return merged_df


#Function to create engine for database
def createengine(name):
    engine = sqlalchemy.create_engine('sqlite:///' + name +'.db')
    return engine


#Function to import data to database
def toSQL(frames,name_table, engine):
    frames.to_sql(name_table, engine, index=False)
    print('Successfully imported data')

## 2. Dow Jones Industrial Average (^DJI)

### Scrape ticker lists from Wikipedia

In [3]:
#Download tickers of ^DJI on wikipedia website
wiki = 'https://en.wikipedia.org/wiki/'

tickerDOW = pd.read_html(wiki+'Dow_Jones_Industrial_Average')[1].Symbol.to_list()

### Using function to download data

In [4]:
USDOWJONES = getData(tickerDOW)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Since different ticker have different dataframes, I will create symbol column and merge data into one dataframes

In [5]:
new_USDOWJONES = merge_data(USDOWJONES, tickerDOW)

In [6]:
new_USDOWJONES

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,1962-01-02,0.000000,4.250000,4.125000,4.156250,0.754036,212800,MMM
1,1962-01-03,0.000000,4.187500,4.085938,4.187500,0.759705,422400,MMM
2,1962-01-04,0.000000,4.257813,4.187500,4.187500,0.759705,212800,MMM
3,1962-01-05,0.000000,4.171875,4.062500,4.078125,0.739862,315200,MMM
4,1962-01-08,0.000000,4.085938,4.031250,4.054688,0.735611,334400,MMM
...,...,...,...,...,...,...,...,...
344591,2022-09-12,137.080002,138.250000,136.970001,138.070007,138.070007,4761500,WMT
344592,2022-09-13,136.860001,137.949997,134.809998,135.220001,135.220001,5895800,WMT
344593,2022-09-14,135.630005,136.529999,134.100006,134.919998,134.919998,6644900,WMT
344594,2022-09-15,135.050003,135.149994,132.970001,133.470001,133.470001,6729800,WMT


Check if the data downloaded is correct or not

In [7]:
new_USDOWJONES.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,1962-01-02,0.0,4.250000,4.125000,4.156250,0.754036,212800,MMM
1,1962-01-03,0.0,4.187500,4.085938,4.187500,0.759705,422400,MMM
2,1962-01-04,0.0,4.257813,4.187500,4.187500,0.759705,212800,MMM
3,1962-01-05,0.0,4.171875,4.062500,4.078125,0.739862,315200,MMM
4,1962-01-08,0.0,4.085938,4.031250,4.054688,0.735611,334400,MMM


In [8]:
yf.download('MMM', start='1962-01-02', end='1962-01-08').reset_index()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,1962-01-02,0.0,4.250000,4.125000,4.156250,0.754035,212800
1,1962-01-03,0.0,4.187500,4.085938,4.187500,0.759705,422400
2,1962-01-04,0.0,4.257813,4.187500,4.187500,0.759705,212800
3,1962-01-05,0.0,4.171875,4.062500,4.078125,0.739862,315200


## 3. Download TSX60 -  a stock market index of 60 large companies listed on the Toronto Stock Exchange
Summary: With Dow Jones Index, we do not have any problem when downloading data. But with TSX60, we will have error when using tickers scraped from Wikipedia to download data from Yahoo Finance API because:
1. There are some tickers that have '.' instead of '-', for example BAM.A should be BAM-A
2. TSX60 tickers have '.TO' at the end

I am going to fix these issues

### Scrape ticker lists from Wikipedia

In [9]:
tickerTSX = pd.read_html(wiki+'S%26P/TSX_60')[0].Symbol.to_list()

In [10]:
tickerTSX[:10]

['AEM', 'AQN', 'ATD', 'BCE', 'BMO', 'BNS', 'ABX', 'BHC', 'BAM.A', 'BIP.UN']

### Cleaning data for ticker lists to meet Yahoo! Finance's API standard

After downloading and manually checking, I realize that the ticker number 39 is incorrect

In [11]:
#Check the tickers are incorrect
for i in range(len(tickerTSX)):
    if type(tickerTSX[i]) != str:
        print(tickerTSX[i], i, type(tickerTSX[i]))

nan 39 <class 'float'>


In [12]:
#Correct the incorrect ticker
tickerTSX[39] = 'NA'
tickerTSX[39]

'NA'

There are some tickers that have '.' instead of '-', and need to have '.TO' at the end.

In [13]:
for ticker in tickerTSX:
    if '.' in str(ticker):
        print(ticker)

BAM.A
BIP.UN
CCL.B
GIB.A
CAR.UN
CTC.A
RCI.B
SJR.B
TECK.B


In [14]:
# I tried to download using this ticker but it failed
yf.download('CTC.A')['Close']

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CTC.A: No data found, symbol may be delisted


Series([], Name: Close, dtype: float64)

In [15]:
#The correct ticker has '-' and '.TO'
yf.download('CTC-A.TO')['Close']

[*********************100%***********************]  1 of 1 completed


Date
1986-01-02     11.750000
1986-01-03     11.630000
1986-01-06     11.630000
1986-01-07     11.750000
1986-01-08     11.630000
                 ...    
2022-09-12    163.410004
2022-09-13    162.130005
2022-09-14    157.570007
2022-09-15    156.580002
2022-09-16    157.759995
Name: Close, Length: 9228, dtype: float64

In [16]:
#I am going to replace "." to "-" and add ".TO" at the end of ticker
newtickerTSX = []
for ticker in tickerTSX:
    if '.' in ticker:
        dot_index = ticker.rfind('.')
        newticker = ticker[:dot_index] + '-' + ticker[dot_index +1:] + '.TO'
        newtickerTSX.append(newticker)
    else:
        newtickerTSX.append(str(ticker) + '.TO')  

### Download historical data from from Yahoo! Finance's API

In [17]:
# Download historical price data
TSX60 = getData(newtickerTSX)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [18]:
new_TSX60 = merge_data(TSX60, newtickerTSX)
new_TSX60.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,1995-01-12,14.88,15.00,14.50,14.50,11.853486,147100,AEM.TO
1,1995-01-13,14.38,14.50,14.13,14.25,11.649120,26500,AEM.TO
2,1995-01-16,14.25,14.25,13.88,13.88,11.346651,3800,AEM.TO
3,1995-01-17,14.25,14.25,13.88,13.88,11.346651,18500,AEM.TO
4,1995-01-18,14.13,14.50,14.13,14.25,11.649120,18500,AEM.TO


In [19]:
new_TSX60.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
342551,2022-09-12,43.970001,44.220001,43.430000,43.820000,43.820000,1016700,WPM.TO
342552,2022-09-13,42.840000,43.820000,42.560001,42.849998,42.849998,954900,WPM.TO
342553,2022-09-14,43.150002,43.689999,42.869999,43.029999,43.029999,732500,WPM.TO
342554,2022-09-15,42.669998,43.060001,41.290001,41.900002,41.900002,1029800,WPM.TO
342555,2022-09-16,41.419998,43.160000,41.340000,42.610001,42.610001,2499900,WPM.TO


## 4. Download fundamental data from Yahoo! Finance's API

In [42]:
def download_fundamental(tickers):
    df = pd.DataFrame()
    for ticker in tickers:
        var = yf.Ticker(ticker).info
        frame = pd.DataFrame([var])
        df = pd.concat([df, frame], ignore_index=True)
    return df

#### Download TSX60 fundamental data

In [39]:
TSXFundamental = download_fundamental(newtickerTSX)

In [43]:
# Looking at Fundamental dataset
TSXFundamental.shape

(60, 159)

We have 60 rows and 159 columns

In [44]:
TSXFundamental.head()

,zip,sector,fullTimeEmployees,longBusinessSummary,city,state,country,companyOfficers,website,maxAge,...,dividendYield,bidSize,dayHigh,coinMarketCapLink,regularMarketPrice,preMarketPrice,logo_url,lastDividendDate,impliedSharesOutstanding,phone
0,M5C 2Y7,Basic Materials,6810.0,Agnico Eagle Mines Limited engages in the expl...,Toronto,ON,Canada,[],https://www.agnicoeagle.com,1,...,0.0371,None,55.89,None,55.33,None,https://logo.clearbit.com/agnicoeagle.com,NaN,NaN,NaN
1,L6J 2X1,Utilities,3445.0,"Algonquin Power & Utilities Corp., through its...",Oakville,ON,Canada,[],https://algonquinpower.com,1,...,0.0527,0,17.78,None,17.53,None,https://logo.clearbit.com/algonquinpower.com,1.656461e+09,0.0,NaN
2,H7L 0E3,Consumer Cyclical,122000.0,Alimentation Couche-Tard Inc. operates and lic...,Laval,QC,Canada,[],https://corpo.couche-tard.com,1,...,0.0074,None,59.11,None,59.08,None,https://logo.clearbit.com/corpo.couche-tard.com,1.662509e+09,0.0,NaN
3,H3E 3B3,Communication Services,49781.0,"BCE Inc., a telecommunications and media compa...",Verdun,QC,Canada,[],https://www.bce.ca,1,...,0.0591,0,62.38,None,62.29,None,https://logo.clearbit.com/bce.ca,1.663114e+09,0.0,NaN
4,H2Y 1L6,Financial Services,43863.0,Bank of Montreal provides diversified financia...,Montreal,QC,Canada,[],https://www.bmo.com,1,...,0.0439,0,127.20,None,126.76,None,https://logo.clearbit.com/bmo.com,1.659053e+09,0.0,NaN


In [45]:
TSXFundamental.tail()

,zip,sector,fullTimeEmployees,longBusinessSummary,city,state,country,companyOfficers,website,maxAge,...,dividendYield,bidSize,dayHigh,coinMarketCapLink,regularMarketPrice,preMarketPrice,logo_url,lastDividendDate,impliedSharesOutstanding,phone
55,V6B 0M3,Communication Services,90800.0,"TELUS Corporation, together with its subsidiar...",Vancouver,BC,Canada,[],https://www.telus.com,1,...,0.0459,0,28.83,None,28.77,None,https://logo.clearbit.com/telus.com,NaN,NaN,NaN
56,M5H 2R2,Industrials,24400.0,Thomson Reuters Corporation provides business ...,Toronto,ON,Canada,[],https://www.tr.com,1,...,0.0160,0,147.30,None,146.88,None,https://logo.clearbit.com/tr.com,NaN,NaN,NaN
57,M5K 1A2,Financial Services,91993.0,"The Toronto-Dominion Bank, together with its s...",Toronto,ON,Canada,[],https://www.td.com,1,...,0.0428,0,87.99,None,87.93,None,https://logo.clearbit.com/td.com,NaN,NaN,NaN
58,L4H 4G3,Industrials,19998.0,"Waste Connections, Inc. provides non-hazardous...",Woodbridge,ON,Canada,[],https://www.wasteconnections.com,1,...,0.0068,0,193.12,None,192.20,None,https://logo.clearbit.com/wasteconnections.com,NaN,NaN,NaN
59,V6E 0C3,Basic Materials,44.0,"Wheaton Precious Metals Corp., a streaming com...",Vancouver,BC,Canada,[],https://www.wheatonpm.com,1,...,0.0172,None,43.16,NaN,42.61,None,https://logo.clearbit.com/wheatonpm.com,NaN,NaN,NaN


In [46]:
TSXFundamental.isnull().sum()

zip                          0
sector                       0
fullTimeEmployees           11
longBusinessSummary          0
city                         0
                            ..
preMarketPrice              60
logo_url                     0
lastDividendDate            48
impliedSharesOutstanding    47
phone                       53
Length: 159, dtype: int64

We have some columns that do not have values. We will drop all columns that have null values

In [47]:
# Select columns that we will use for future data analysis and visualization

select_columns = ['symbol', 'shortName','longName', 'sector', 'longBusinessSummary', 'city', 'state', 'country', 'website','industry','ebitdaMargins',
       'profitMargins', 'grossMargins', 'operatingCashflow',
       'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice',
       'recommendationKey', 'grossProfits', 'freeCashflow',
       'targetMedianPrice', 'currentPrice', 'earningsGrowth',
       'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions',
       'targetMeanPrice', 'debtToEquity', 'returnOnEquity',
       'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue',
       'totalCashPerShare', 'financialCurrency', 'revenuePerShare',
       'market',
       ]

In [48]:
subTSX_Fundamental = TSXFundamental[select_columns]

In [52]:
subTSX_Fundamental.head()

,symbol,shortName,longName,sector,longBusinessSummary,city,state,country,website,industry,...,debtToEquity,returnOnEquity,targetHighPrice,totalCash,totalDebt,totalRevenue,totalCashPerShare,financialCurrency,revenuePerShare,market
0,AEM.TO,AGNICO EAGLE MINES LIMITED,Agnico Eagle Mines Limited,Basic Materials,Agnico Eagle Mines Limited engages in the expl...,Toronto,ON,Canada,https://www.agnicoeagle.com,Gold,...,9.684,0.05365,133.86,1006854976,1575686016,4814681088,2.227,USD,14.516,ca_market
1,AQN.TO,ALGONQUIN POWER AND UTILITIES C,Algonquin Power & Utilities Corp.,Utilities,"Algonquin Power & Utilities Corp., through its...",Oakville,ON,Canada,https://algonquinpower.com,Utilities—Renewable,...,102.899,0.01318,23.69,84997000,7560878080,2483369984,0.125,USD,3.784,ca_market
2,ATD.TO,ALIMENTATION COUCHE-TARD INC,Alimentation Couche-Tard Inc.,Consumer Cyclical,Alimentation Couche-Tard Inc. operates and lic...,Laval,QC,Canada,https://corpo.couche-tard.com,Specialty Retail,...,75.368,0.22438,89.51,2217600000,9359399936,67888701440,2.170,USD,64.647,ca_market
3,BCE.TO,BCE INC.,BCE Inc.,Communication Services,"BCE Inc., a telecommunications and media compa...",Verdun,QC,Canada,https://www.bce.ca,Telecom Services,...,127.604,0.13147,75.00,596000000,30325000192,23755999232,0.654,CAD,26.121,ca_market
4,BMO.TO,BANK OF MONTREAL,Bank of Montreal,Financial Services,Bank of Montreal provides diversified financia...,Montreal,QC,Canada,https://www.bmo.com,Banks—Diversified,...,NaN,0.17985,161.51,335155003392,176752001024,29721999360,496.996,CAD,45.240,ca_market


In [53]:
subTSX_Fundamental.tail()

,symbol,shortName,longName,sector,longBusinessSummary,city,state,country,website,industry,...,debtToEquity,returnOnEquity,targetHighPrice,totalCash,totalDebt,totalRevenue,totalCashPerShare,financialCurrency,revenuePerShare,market
55,T.TO,TELUS CORPORATION,TELUS Corporation,Communication Services,"TELUS Corporation, together with its subsidiar...",Vancouver,BC,Canada,https://www.telus.com,Telecom Services,...,131.334,0.11103,36.00,774000000,21543999488,17072000000,0.560,CAD,12.502,ca_market
56,TRI.TO,THOMSON REUTERS CORPORATION,Thomson Reuters Corporation,Industrials,Thomson Reuters Corporation provides business ...,Toronto,ON,Canada,https://www.tr.com,Specialty Business Services,...,27.767,0.11358,173.03,703000000,4060999936,6441999872,1.443,USD,13.117,ca_market
57,TD.TO,TORONTO-DOMINION BANK,The Toronto-Dominion Bank,Financial Services,"The Toronto-Dominion Bank, together with its s...",Toronto,ON,Canada,https://www.td.com,Banks—Diversified,...,NaN,0.15336,100.73,543247990784,357727010816,44033998848,301.152,CAD,24.244,ca_market
58,WCN.TO,WASTE CONNECTIONS INC,"Waste Connections, Inc.",Industrials,"Waste Connections, Inc. provides non-hazardous...",Woodbridge,ON,Canada,https://www.wasteconnections.com,Waste Management,...,89.375,0.09321,227.92,391416992,6040936960,6401674240,1.522,USD,24.600,ca_market
59,WPM.TO,WHEATON PRECIOUS METALS CORP,Wheaton Precious Metals Corp.,Basic Materials,"Wheaton Precious Metals Corp., a streaming com...",Vancouver,BC,Canada,https://www.wheatonpm.com,Gold,...,0.042,0.12328,51.05,376163008,2698000,1184790016,0.833,USD,2.630,ca_market


#### Download Dow Jones's Fundamental Data

In [54]:
DOWFundamental = download_fundamental(tickerDOW)

In [55]:
subDOWFundamental = DOWFundamental[select_columns]

# 4. Download market historical prices

In [57]:
index_list = ['TX60.TS', '^DJI', '^GSPC','^IXIC']

In [59]:
index_price_data = getData(index_list)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [60]:
merge_index_price_data = merge_data(index_price_data, index_list)

In [61]:
merge_index_price_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2022-09-16,1171.530029,1174.510010,1166.209961,1172.619995,1172.619995,294512587,TX60.TS
1,1992-01-02,3152.100098,3172.629883,3139.310059,3172.399902,3172.399902,23550000,^DJI
2,1992-01-03,3172.399902,3210.639893,3165.919922,3201.500000,3201.500000,23620000,^DJI
3,1992-01-06,3201.500000,3213.330078,3191.860107,3200.100098,3200.100098,27280000,^DJI
4,1992-01-07,3200.100098,3210.199951,3184.479980,3204.800049,3204.800049,25510000,^DJI


In [62]:
merge_index_price_data.shape

(39051, 8)

# 5. Create database & import data into DJIA table

In [63]:
# Create engine 
stockengine = createengine('StockDatabase')

In [64]:
#Create table for DJIA Historical Prices
toSQL(new_USDOWJONES, "DJIA_Historical_Data", stockengine)

Successfully imported data


In [65]:
#Create table for TSX60 Historical Prices
toSQL(new_TSX60, "TSX60_Historical_Data", stockengine)

Successfully imported data


In [66]:
toSQL(subTSX_Fundamental, "TSX60_Fundamental", stockengine)

Successfully imported data


In [67]:
toSQL(subDOWFundamental, "DOW_Fundamental", stockengine)

Successfully imported data


In [68]:
toSQL(merge_index_price_data, "Markets", stockengine)

Successfully imported data


In [69]:
'CNQ.TO' in newtickerTSX

True